In [1]:
from pyspark.sql import SparkSession
import re
import pandas as pd
import os

In [2]:
# Inicializa a sessão do Spark
spark = SparkSession.builder.appName("Contagem de Palavras").getOrCreate()


23/06/22 19:56:53 WARN Utils: Your hostname, WNL900TI148 resolves to a loopback address: 127.0.1.1; using 192.168.100.66 instead (on interface wlp0s20f3)
23/06/22 19:56:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/22 19:56:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Caminho do arquivo de texto
path = "../data/"
words, counts, sites = [], [], []

df_termos = pd.read_csv('df_keywords.csv')
termos = list(df_termos.word.values)
rsociais_files = ['cnnviagemegastronomia.txt', 'reddit.txt']

for file_name in os.listdir(path):
    if file_name.endswith('.txt') and file_name not in rsociais_files:
        file_path = path+file_name
        print(file_name)

        # Carrega o arquivo como um DataFrame
        texto = spark.read.text(file_path)

        # Realiza a contagem das expressões
        for termo in termos:
            pattern = r'\b{}\b'.format(re.escape(termo))  # Cria um padrão regex para expressão exata
            count = texto.filter(texto.value.rlike(pattern)).count()
            words.append(termo)
            counts.append(count)
            sites.append(str(file_name)[:-4])

conteudo_uol_cozinha-por-ai.txt
conteudo_Suzana_Barelli.txt
blogdosvinhos.txt
conteudo_o-cachacier.txt
conteudo_So_queijo.txt
conteudo_uol_timtim.txt
guiadasemana.txt
conteudo_Blog_da_Belle.txt
conteudo_um-cafe-para-dividir.txt
conteudo_uol_vai-ter-churras.txt
vinhosemsegredo.txt
conteudo_blog-da-patricia-ferra.txt
conteudo_uol_ao-ponto.txt
vinhoegastronomia.txt
conteudo_Balcão_do_Giba.txt
revista_menu.txt
conteudo_vinhonosso.txt
dataset_colunistas_vinho.txt


In [7]:
df = pd.DataFrame({'word': words, 'count': counts, 'site': sites})
df = df[df['count']>0].reset_index(drop=True)
df = df.sort_values(by='count', ascending=False)
df.to_csv('expression_count.csv', index=False)

In [8]:
df.head()

,word,count,site
119,Pinot Noir,416,dataset_colunistas_vinho
134,vinho branco,182,dataset_colunistas_vinho
73,frutas vermelhas,122,vinhoegastronomia
150,bons vinhos,110,dataset_colunistas_vinho
118,vinho tinto,98,dataset_colunistas_vinho
